In [5]:
# Класс MasterNode для хранения информации о ноде
class MasterNode:
    def __init__(self, num_node, start_arr, stop_arr, status="зарегистрировано"):
        """
        Создает структуру хранения информации о ноде.

        Вход:
        - num_node: int, номер ноды
        - start_arr: int, начальный индекс диапазона строк массива
        - stop_arr: int, конечный индекс диапазона строк массива
        - status: str, статус ноды ("зарегистрировано", "В процессе", "Выполнено")
        """
        self.num_node = num_node
        self.start_arr = start_arr
        self.stop_arr = stop_arr
        self.status = status

    def __repr__(self):
        return (f"MasterNode(num_node={self.num_node}, start_arr={self.start_arr}, "
                f"stop_arr={self.stop_arr}, status={self.status})")


# Класс Task для хранения информации о задаче
class Task:
    def __init__(self, user_id, task_id, nodes):
        """
        Создает структуру хранения информации о задаче.

        Вход:
        - user_id: int, ID пользователя
        - task_id: int, ID задачи
        - nodes: list, список объектов MasterNode, связанных с задачей
        """
        self.user_id = user_id
        self.task_id = task_id
        self.nodes = nodes
        self.task_status = self.calculate_task_status()  # Статус задачи вычисляется на основе нод

    def calculate_task_status(self):
        """
        Вычисляет статус задачи на основе статусов нод.
        """
        if all(node.status == "Выполнено" for node in self.nodes):
            return "Выполнено"
        elif any(node.status == "В процессе" for node in self.nodes):
            return "В процессе"
        else:
            return "зарегистрировано"

    def __repr__(self):
        return (f"Task(user_id={self.user_id}, task_id={self.task_id}, "
                f"task_status={self.task_status}, nodes={self.nodes})")


# Класс TaskAdmin для управления задачами
class TaskAdmin:
    def __init__(self):
        """
        Создает структуры хранения информации о задачах.
        """
        self.tasks = {}  # Словарь для хранения задач, где ключ - task_id, значение - объект Task

    def create_task(self, user_id, task_id, nodes):
        """
        Создает новую задачу.

        Вход:
        - user_id: int, ID пользователя
        - task_id: int, ID задачи
        - nodes: list, список объектов MasterNode, связанных с задачей

        Действие:
        - Создает объект Task и добавляет его в self.tasks.
        """
        task = Task(user_id, task_id, nodes)
        self.tasks[task_id] = task
        print(f"Задача {task_id} зарегистрирована для пользователя {user_id}.")

    def update_node_status(self, task_id, num_node, status):
        """
        Обновляет статус ноды в задаче.

        Вход:
        - task_id: int, ID задачи
        - num_node: int, номер ноды
        - status: str, статус ноды ("зарегистрировано", "В процессе", "Выполнено")

        Действие:
        - Обновляет статус указанной ноды и пересчитывает статус задачи.
        """
        if task_id in self.tasks:
            task = self.tasks[task_id]
            for node in task.nodes:
                if node.num_node == num_node:
                    node.status = status
                    task.task_status = task.calculate_task_status()  # Пересчитываем статус задачи
                    print(f"Статус ноды {num_node} в задаче {task_id} обновлен на '{status}'.")
                    return
            print(f"Нода {num_node} не найдена в задаче {task_id}.")
        else:
            print(f"Задача {task_id} не найдена.")

    def get_task_status(self, task_id):
        """
        Возвращает текущий статус задачи.

        Вход:
        - task_id: int, ID задачи

        Выход:
        - dict, информация о статусе задачи

        Действие:
        - Возвращает текущий статус задачи и статусы нод.
        """
        if task_id in self.tasks:
            task = self.tasks[task_id]
            return {
                "task_id": task.task_id,
                "user_id": task.user_id,
                "task_status": task.task_status,
                "nodes": [
                    {
                        "num_node": node.num_node,
                        "start_arr": node.start_arr,
                        "stop_arr": node.stop_arr,
                        "status": node.status
                    }
                    for node in task.nodes
                ]
            }
        else:
            print(f"Задача {task_id} не найдена.")
            return None

    def filter_tasks(self, status="зарегистрировано"):
        """
        Фильтрует задачи по статусу.

        Вход:
        - status: str, статус задачи ("зарегистрировано", "В процессе", "Выполнено")

        Выход:
        - list, список задач с указанным статусом

        Действие:
        - Возвращает список задач, соответствующих указанному статусу.
        """
        return [task_id for task_id, task in self.tasks.items() if task.task_status == status]

    def delete_task(self, task_id):
        """
        Удаляет задачу.

        Вход:
        - task_id: int, ID задачи

        Действие:
        - Удаляет задачу из self.tasks.
        """
        if task_id in self.tasks:
            del self.tasks[task_id]
            print(f"Задача {task_id} удалена.")
        else:
            print(f"Задача {task_id} не найдена.")

    def to_scheduler(self, task_id):
        """
        Отправляет информацию о выполненных диапазонах массива в Scheduler.

        Вход:
        - task_id: int, ID задачи

        Действие:
        - Отправляем количество нод через scheduler.nodes().
        - Отправляем информацию о выполненных строках через scheduler.free_node().
        - Отправляем информацию о новой зарегистрированной задаче через scheduler.new_node().
        """
        if task_id in self.tasks:
            task = self.tasks[task_id]

            # Отправляем количество нод
            scheduler.nodes(len(task.nodes))

            # Отправляем информацию о выполненных строках для каждой ноды
            for node in task.nodes:
                if node.status == "Выполнено":
                    scheduler.free_node(node.num_node, node.stop_arr - node.start_arr + 1)

            # Отправляем информацию о новой зарегистрированной задаче
            scheduler.new_node(task.task_id, task.user_id, task.nodes)
        else:
            print(f"Задача {task_id} не найдена.")

In [7]:
# Создаем ноды (теперь MasterNode)
node1 = MasterNode(num_node=1, start_arr=1, stop_arr=20)
node2 = MasterNode(num_node=2, start_arr=21, stop_arr=40)

# Инициализируем TaskAdmin
task_admin = TaskAdmin()

# Создаем задачу
task_admin.create_task(user_id=1, task_id=1, nodes=[node1, node2])

# Обновляем статус ноды
task_admin.update_node_status(task_id=1, num_node=1, status="Выполнено")

# Получаем статус задачи
status = task_admin.get_task_status(task_id=1)
print(status)

# Фильтруем задачи
filtered_tasks = task_admin.filter_tasks(status="зарегистрировано")
print(filtered_tasks)

# Удаляем задачу
task_admin.delete_task(task_id=1)

# Отправляем информацию в Scheduler
task_admin.to_scheduler(task_id=1)

Задача 1 зарегистрирована для пользователя 1.
Статус ноды 1 в задаче 1 обновлен на 'Выполнено'.
{'task_id': 1, 'user_id': 1, 'task_status': 'зарегистрировано', 'nodes': [{'num_node': 1, 'start_arr': 1, 'stop_arr': 20, 'status': 'Выполнено'}, {'num_node': 2, 'start_arr': 21, 'stop_arr': 40, 'status': 'зарегистрировано'}]}
[1]
Задача 1 удалена.
Задача 1 не найдена.
